# Fig. 1

In [1]:
import numpy as np
import networkx as nx
from dwdynamics import helpers ,instance
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import linregress
import dwave_networkx as dnx
import pandas as pd
import numpy as np
from matplotlib.lines import Line2D
import dimod
import math
import minorminer

helpers.set_pub_style(scale=1.0, fontsize=18)

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle

fig = plt.figure(figsize=(16, 8))
gs = gridspec.GridSpec(2, 4, height_ratios=[1, 1.2], hspace=0.4, wspace=0.3, width_ratios=[1.2, 1, 1, 1.2])

ax1 = fig.add_subplot(gs[0, 0:2])
ax2 = fig.add_subplot(gs[0, 2:4])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1:3])
ax5 = fig.add_subplot(gs[1, 3])

# axes = [ax1, ax2, ax3, ax4, ax5]
# for ax in axes:
  # ax.axis('off')

ax3.text(0.5, 1.1, '(a)', ha='center', va='center', fontsize=16, transform=ax1.transAxes)
ax3.text(0.5, 1.1, '(b)', ha='center', va='center', fontsize=16, transform=ax2.transAxes)
ax3.text(0.5, 1.1, '(c)', ha='center', va='center', fontsize=16, transform=ax3.transAxes)
ax4.text(0.7, 1.1, '(d)', ha='center', va='center', fontsize=16, transform=ax4.transAxes)

######################### ax1 - schematic of time evolution #########################
ax = ax1
ax.axis('off')

x0, y0 = 0.2, 0.5  # psi_0
x1, y1 = 2.0, 0.5  # H(t) box
x2, y2 = 3.8, 0.5  # psi(t)

# Draw psi_0
ax.text(x0, y0, r"$|\psi_0\rangle$", fontsize=18, va='center', ha='center')

# Draw H(t) box
box_width = 1.0
box_height = 0.5
H_box = Rectangle((x1 - box_width/2, y1 - box_height/2), box_width, box_height,
                  edgecolor='black', facecolor='white')
ax.add_patch(H_box)
ax.text(x1, y1, r"$H(t)$", fontsize=18, va='center', ha='center')

# Draw psi(t)
ax.text(x2, y2, r"$|\psi(t)\rangle$", fontsize=18, va='center', ha='center')

# Arrows (shorter)
ax.annotate("", xy=(x1 - box_width/2 - 0.05, y1), xytext=(x0 + 0.25, y0),
            arrowprops=dict(arrowstyle="->", lw=1.2))
ax.annotate("", xy=(x2 - 0.25, y2), xytext=(x1 + box_width/2 + 0.05, y1),
            arrowprops=dict(arrowstyle="->", lw=1.2))

# Schrödinger equation above
ax.text(x1 + 0.1, y1 + 0.8, r"$\frac{\partial}{\partial t}|\psi(t)\rangle = H(t)|\psi(t)\rangle$",
        fontsize=20, va='center', ha='center')

# Time evolution label
ax.text(x1, y1 - 0.65, "Time evolution", fontsize=18, va='center', ha='center')

# Adjust limits tightly to content
ax.set_xlim(0.2, 4.3)
ax.set_ylim(-0.2, 1.6)

################################# ax2 - schematic of sliced propagator #########################
ax = ax2
# Czas i fala
t = np.linspace(0, 4, 1000)
psi = np.cos(np.pi * t / 2)  # fala zaczyna w 1, potem 0, -1, 0, 1

# Hamiltonian: zmienia się co 1 jednostkę czasu
hamiltonian_values = [1, 0, -1, 0]
color_map = {1: '#ffcccc', 0: '#99bbdd', -1: '#cceeff'}
t_strobe = np.arange(0, 5)

ax.plot(t, psi, color='black', linewidth=1.5, label=r'$\psi(t)$')
# Kolorowe tła i ewolucje jednostkowe
for i in range(len(t_strobe) - 1):
    x_start = t_strobe[i]
    x_end = t_strobe[i+1]
    h_val = hamiltonian_values[i]
    ax.axvspan(x_start, x_end, color=color_map[h_val], alpha=0.5)
    ax.axvline(x_start, color='gray', linestyle='--', linewidth=0.7)

    y_pos = -1.4 + 0.1 * (i % 2)
    ax.annotate('', xy=(x_end, y_pos), xytext=(x_start, y_pos),
                arrowprops=dict(arrowstyle='->', color='black', linewidth=1))
    ax.text((x_start + x_end)/2, y_pos +0.15,
            r'$U(t_{%d}, t_{%d})$' % (i+1, i), ha='center', fontsize=18)

# Ostatnia linia
ax.axvline(t_strobe[-1], color='gray', linestyle='--', linewidth=0.7)

# Punkty stroboskopowe
psi_strobe = np.cos(np.pi * t_strobe / 2)
ax.plot(t_strobe, psi_strobe, 'o', color='black')

# Opisy osi
ax.set_xlabel(r'Time step $t_n$', fontsize=18)
ax.set_ylabel(r'Expectation value $\langle \sigma_z \rangle$', fontsize=18)
ax.set_xticks(np.arange(0, 5, 1))
ax.set_yticks([-1, 0, 1])
ax.set_ylim(-1.5, 1.5)
ax.set_xlim(-0.05, 4.05)

######################### ax3 - schematic of clock hamiltonian #########################
ax = ax3
ax.axis('off')
# Matrix entries using LaTeX for math formatting
matrix = [
    [r"$2\hat{I}$", r"$\hat{U}^\dagger(t_1,t_0)$", r"$0$", r"$0$"],
    [r"$\hat{U}(t_1,t_0)$", r"$2\hat{I}$",r"$\hat{U}^\dagger(t_2,t_1)$" , r"$0$"],
    [r"$0$", r"$\hat{U}(t_2,t_1)$", r"$2\hat{I}$", r"$\hat{U}^\dagger(t_3,t_2)$"],
    [r"$0$", r"$0$", r"$\hat{U}(t_3,t_2)$", r"$\hat{I}$"]
]

colors = [
    ["#f4c2ff", "#ffcccc", "#ffffff", "#ffffff"],
    ["#ffcccc", "#f4c2ff", "#99bbdd", "#ffffff"],
    ["#ffffff", "#99bbdd", "#f4c2ff", "#cceeff"],
    ["#ffffff", "#ffffff", "#cceeff", "#f4c2ff"]
]

ax.set_xlim(0, 4)
ax.set_ylim(0, 4)

# Draw cells with colors and LaTeX text
for i in range(4):
    for j in range(4):
        rect = Rectangle((j, 3 - i), 1, 1, facecolor=colors[i][j],
                         edgecolor='black', linewidth=1.5)
        ax.add_patch(rect)
        ax.text(j + 0.5, 3 - i + 0.5, matrix[i][j],
                ha='center', va='center', fontsize=12)
ax.text(2, -0.5, r"Clock Hamiltonian $C$", fontsize=18, ha='center')
############################ ax4 - diagram ##########################
ax = ax4
ax.axis('off')
from matplotlib.patches import FancyArrowPatch
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from pdf2image import convert_from_path
from PIL import Image

# Load and resize PNG images
dwave_img = Image.open("../plots/dwave.jpg")
velox_img = Image.open("../plots/gpu.png")

# Convert PDFs to images
input_img = convert_from_path("../plots/input.pdf", dpi=150)[0]


# Arrows with clip_on=False and padding for visibility
arrow1 = FancyArrowPatch((3.5, 2.1), (10, 2.7), connectionstyle="arc3,rad=0.2",
                         arrowstyle='-|>', mutation_scale=20, color="blue",
                         linewidth=2, clip_on=False)
arrow2 = FancyArrowPatch((3.5, 1.9), (10, 1.3), connectionstyle="arc3,rad=-0.2",
                         arrowstyle='-|>', mutation_scale=20, color="green",
                         linewidth=2, clip_on=False)
ax.add_patch(arrow1)
ax.add_patch(arrow2)

# Labels
ax.text(3.8, 2.7, "D-Wave QPU", fontsize=16, color="blue", weight='bold')
ax.text(3.8, 1.1, "VeloxQ", fontsize=16, color="green", weight='bold')

# Input image
input_box = OffsetImage(input_img, zoom=0.34)
input_ab = AnnotationBbox(input_box, (1.2, 2), frameon=False, pad=0.3)
ax.add_artist(input_ab)


# D-Wave image (small, under arrow)
dwave_box = OffsetImage(dwave_img, zoom=0.1)
dwave_ab = AnnotationBbox(dwave_box, (8, 3.3), frameon=False, pad=0.1)
ax.add_artist(dwave_ab)

# # Velox GPU image (small, under arrow)
velox_box = OffsetImage(velox_img, zoom=0.2)
velox_ab = AnnotationBbox(velox_box, (7.9, 0.7), frameon=False, pad=0.1)
ax.add_artist(velox_ab)

# Set limits to ensure clean framing
ax.set_xlim(0, 10.5)
ax.set_ylim(0, 4)

############################ ax5 - example dynamics #########################
ax = ax5
import qutip as qp

system = 3

tp_pairs = [(3,3)]
# We simulate evolution under the Hamiltonian 1/2 pi * Sigma_y...

# Loop over all rows in velox_results and plot all dynamics on the same panel,
# coloring each curve according to its best_energy.
colors = ["r", "g", "b", "c", "m", "y", "k"]
markers = ["o", "s", "^", "D", "v", "<", ">"]

SY = np.array([[0, -1.0j], [1.0j, 0.0]])
PSI0 = np.array([1, 0], dtype=np.complex128)  # Difference b. Initial state can now be complex.

system_latex_dict = {
    1: 'H = $\\frac{\\pi}{2}\\sigma_y$',
    3: 'one qubit pt symmetric system (seed=1)',
    4: 'maximally entangled 2 qubit system'
}

for i, (timepoints,precision) in enumerate(tp_pairs):

    s = helpers.get_dwave_sample_set(system, timepoints)
    inst = instance.Instance(system)
    inst.create_instance(precision, timepoints)
    H = inst.H
    PSI0 = inst.psi0
    dim = int(math.log2(H.shape[0]))    #dim=2

    print(H.shape)
    


    #psi_0 = qp.tensor(qp.basis(2, 0), qp.basis(2, 1))
    times = np.linspace(0, timepoints-1, 100)
    baseline = qp.mesolve(qp.Qobj(H), qp.Qobj(PSI0), times, e_ops=[qp.sigmaz()]).expect
    
    ax.plot(times, baseline[0], "k--",label="|00>")

    problem = inst.problem

    sample = s.first.sample
    sample_energy = s.first.energy
    vec =problem.interpret_sample(sample)

    expect_00 = [(state.conj()@ qp.sigmaz().full() @ state).real for state in vec]

    ax.plot(problem.times, expect_00,label=f"|00> Energy: {round(sample_energy,4)}", marker='o',markersize=15,linestyle='None',alpha=1.0,color='red')
  #  ax.scatter(problem.times, expect_11, color='black', marker=markers[j % len(markers)],label=f"|11> Energy: {sample_energy,4)}", alpha=1.0, s=100)

        # ax.plot(problem.times, expect, color=colors[j % len(colors)], alpha=0.5, linewidth=0.1)
    ax.set_xlabel(r"Time step $t_n$", fontsize=18)
    ax.set_ylabel(r"Expectation value $\langle \sigma_z \rangle$", fontsize=18)
#fig.suptitle(f"Advantage {topology} " +rf"evolution of {system} ",fontsize=16)
ax.set_xticks(np.arange(0, timepoints, 1))  # Show all integer timepoints as xticks

plt.savefig("../plots/fig1.pdf", bbox_inches='tight', dpi=300)
plt.show()

0.24999999999999678
0.24999999999999722
0.24999999999999922
0.249999999999998
0.24999999999999722
0.12499999999999856
0.24999999999999745
0.24999999999999623
0.2499999999999981
0.2499999999999981
0.24999999999999678
0.249999999999999
0.24999999999999756
(2, 2)
